<a href="https://colab.research.google.com/github/yinpu/deep_learning_study/blob/master/MFFRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MFFRM 

## 数据预处理
* book -->book-crossing数据集
* music -->last.fm数据集
* movie -->movielen数据集

In [ ]:
#数据预处理函数
import numpy as np
import os
import  tensorflow as tf

RATING_FILE_NAME={
    'movie': 'ratings.dat',
    'book':'BX-Book-Ratings.csv',
    'music':'user_artists.dat'
}
SEP={
    'movie':'::',
    'book':';',
    'music':'\t'
}
THRESHOLD={
    'movie':4,
    'book':0,
    'music':0
}
DATASETS={'movie','book','music'}

def read_item_index2entity_id_file():
    file = './data/'+DATASET+'/item_index2entity_id.txt'
    i=0
    for line in open(file,encoding='utf-8').readlines():
        item_index=line.strip().split('\t')[0]
        satori_id=line.strip().split('\t')[1]
        item_index_old2new[item_index]=i
        entity_id2index[satori_id]=i
        i+=1
        
def convert_rating():
    file = './data/'+DATASET+'/'+RATING_FILE_NAME[DATASET]
    item_set=set(item_index_old2new.values())
    user_pos_rating={}
    user_neg_rating={}
    for line in open(file,encoding='utf-8').readlines():
        array =line.strip().split(SEP[DATASET])
        if DATASET=='book':
            array=list(map(lambda x:x[1:-1],array))
        item_index_old=array[1]
        if item_index_old not in item_index_old2new:
            continue
        item_index=item_index_old2new[item_index_old]
        user_index_old=int(array[0])
        rating=float(array[2])
        if rating >=THRESHOLD[DATASET]:
            if user_index_old not in user_pos_rating:
                user_pos_rating[user_index_old]=set()
            user_pos_rating[user_index_old].add(item_index)
        else:
            if user_index_old not in user_neg_rating:
                user_neg_rating[user_index_old]=set()
            user_neg_rating[user_index_old].add(item_index)
    print('coverting rating file . . .')
    writer=open('data/'+DATASET+'/rating_final.txt','w',encoding='utf-8')
    user_cnt=0
    user_index_old2new={}
    for user_index_old,pos_item_set in user_pos_rating.items():
        if user_index_old not in user_index_old2new:
            user_index_old2new[user_index_old]=user_cnt
            user_cnt+=1
        user_index=user_index_old2new[user_index_old]
        for item in pos_item_set:
            writer.write("%d\t%d\t1\n"%(user_index,item))
        unwatch_set=item_set-pos_item_set
        if user_index_old in user_neg_rating:
            unwatch_set-=user_neg_rating[user_index_old]
        for item in np.random.choice(list(unwatch_set),size=len(pos_item_set),replace=False):
            writer.write("%d\t%d\t0\n"%(user_index,item))
    writer.close()
    print("num of users:",user_cnt)
    print("num of item:",len(item_set))

def convert_kg():
    print('converting kg.txt file')
    entity_cnt=len(entity_id2index)
    relation_cnt=0
    writer=open('data/'+DATASET+'/kg_final.txt','w',encoding='utf-8')
    file=open('data/'+DATASET+'/kg.txt','r',encoding='utf-8')
    for line in file:
        array=line.strip().split('\t')
        head_old=array[0]
        relation_old=array[1]
        tail_old=array[2]
        if head_old not in entity_id2index:
            continue
        head=entity_id2index[head_old]
        if tail_old not in entity_id2index:
            entity_id2index[tail_old]=entity_cnt
            entity_cnt+=1
        tail=entity_id2index[tail_old]
        if relation_old not in relation_id2index:
            relation_id2index[relation_old]=relation_cnt
            relation_cnt+=1
        relation=relation_id2index[relation_old]
        writer.write("%d\t%d\t%d\n"%(head,relation,tail))
    writer.close()
    print('num of entities:',entity_cnt)
    print('num of relations:',relation_cnt)

In [ ]:
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir('/content/drive/Shared drives/yinpu/MyProject/FFMKR')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#数据集处理
for DATASET in DATASETS:
    print(DATASET+"  :   ")
    np.random.seed(555)
    entity_id2index={}
    relation_id2index={}
    item_index_old2new={}
    read_item_index2entity_id_file()
    convert_rating()
    convert_kg()

book  :   
coverting rating file . . .
num of users: 17860
num of item: 14910
converting kg.txt file
num of entities: 24039
num of relations: 10
music  :   
coverting rating file . . .
num of users: 1872
num of item: 3846
converting kg.txt file
num of entities: 9366
num of relations: 60
movie  :   
coverting rating file . . .
num of users: 6036
num of item: 2347
converting kg.txt file
num of entities: 6729
num of relations: 7


## 数据加载

In [ ]:
def load_data(DATASET):
    n_user,n_item,train_data,eval_data,test_data=load_rating(DATASET)
    n_entity,n_relation,kg=load_kg(DATASET)
    print('data loaded')
    return n_user,n_item,n_entity,n_relation,train_data,eval_data,test_data,kg

def load_rating(DATASET):
    print('reading rating file ....')
    rating_file='data/'+DATASET+'/rating_final'
    if os.path.exists(rating_file+'.npy'):
        rating_np=np.load(rating_file+'.npy')
    else:
        rating_np=np.loadtxt(rating_file+'.txt',dtype=np.int32)
        np.save(rating_file+'.npy',rating_np)
    n_user=len(set(rating_np[:,0]))
    n_item=len(set(rating_np[:,1]))
    train_data,eval_data,test_data=dataset_split(rating_np)
    return n_user,n_item,train_data,eval_data,test_data

def dataset_split(rating_np):
    print('splitting dataset ...')
    eval_ratio=0.2
    test_ration=0.2
    n_rating=rating_np.shape[0]
    eval_indices=np.random.choice(list(range(n_rating)),size=int(n_rating*eval_ratio),replace=False)
    left=set(range(n_rating))-set(eval_indices)
    test_indices=np.random.choice(list(left),size=int(n_rating*test_ration),replace=False)
    train_indices=list(left-set(test_indices))
    train_data,eval_data,test_data=rating_np[train_indices],rating_np[eval_indices],rating_np[test_indices]
    return train_data,eval_data,test_data
def load_kg(DATASET):
    print('reading KG file')
    kg_file='data/'+DATASET+'/kg_final'
    if os.path.exists(kg_file + '.npy'):
        kg = np.load(kg_file + '.npy')
    else:
        kg = np.loadtxt(kg_file + '.txt', dtype=np.int32)
        np.save(kg_file + '.npy', kg)
    n_entity=len(set(kg[:,0])|set(kg[:,2]))
    n_relation=len(set(kg[:,1]))
    return n_entity,n_relation,kg

## MFFRM模型实现
MFFRM由以下三部分组成：
* 特征融合单元（仿射变换+交叉压缩单元）
* 推荐模块
* 知识图谱学习特征模块

In [ ]:
#交叉操作
class CrossLayer(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(CrossLayer,self).__init__(**kwargs)
    def call(self, inputs,):
        v,e=inputs
        v=tf.expand_dims(v,axis=2)
        e=tf.expand_dims(e,axis=1)
        c_matrix=v@e
        c_matrix_t=tf.transpose(c_matrix,perm=[0,2,1])
        return c_matrix,c_matrix_t,inputs
    def get_config(self):
        config = {}
        base_config = super(CrossLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

#交叉压缩单元
class CompressLayer(tf.keras.layers.Layer):
    def __init__(self,weight_regularizer,**kwargs):
        super(CompressLayer,self).__init__(**kwargs)
        self.weight_regularizer=tf.keras.regularizers.get(weight_regularizer)
    def build(self, input_shape):
        self.dim=input_shape[0][-1]
        self.kernel=self.add_weight(name='kernel',shape=(self.dim,1),regularizer=self.weight_regularizer)
        self.kernel_t=self.add_weight(name='kernel_t',shape=(self.dim,1),regularizer=self.weight_regularizer)
        self.bias=self.add_weight(name='bias',shape=(self.dim,),initializer=tf.initializers.zeros)
    def call(self, inputs):
        c_matrix,c_matrix_t,(v,e)=inputs
        c_matrix=tf.reshape(c_matrix,shape=(-1,self.dim))
        c_matrix_t=tf.reshape(c_matrix_t,shape=(-1,self.dim))
        return tf.reshape(c_matrix@self.kernel+c_matrix_t@self.kernel_t,shape=(-1,self.dim))+v+e+self.bias
    def get_config(self):
        config = {
            'weight_regularizer': tf.keras.regularizers.serialize(self.weight_regularizer)
        }
        base_config = super(CompressLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

#特征融合单元
def feature_fusion_unit(inputs,weight_regularizer):
    v,e=inputs
    dim=v.shape[-1]
    v=tf.keras.layers.Dense(dim,activation=None,kernel_regularizer=weight_regularizer)(v)
    e=tf.keras.layers.Dense(dim,activation=None,kernel_regularizer=weight_regularizer)(e)
    cross_feature_matrix=CrossLayer()((v,e))
    v_out=CompressLayer(weight_regularizer)(cross_feature_matrix)
    e_out=CompressLayer(weight_regularizer)(cross_feature_matrix)
    return v_out,e_out

In [ ]:
#MFFRM模型
def MFFRM_model(n_user,n_item,n_entity,n_relation,dim=8,L=1,H=1,l2=1e-6,use_inner_product=True):
    l2=tf.keras.regularizers.l2(l2)
    user_id=tf.keras.Input(shape=(),dtype=tf.int32,name='user_id')
    item_id=tf.keras.Input(shape=(),dtype=tf.int32,name='item_id')
    head_id=tf.keras.Input(shape=(),dtype=tf.int32,name='head_id')
    relation_id=tf.keras.Input(shape=(),dtype=tf.int32,name='relation_id')
    tail_id=tf.keras.Input(shape=(),dtype=tf.int32,name='tail_id')
    user_embedding=tf.keras.layers.Embedding(input_dim=n_user,output_dim=dim,
                                             embeddings_regularizer=l2)
    item_embedding=tf.keras.layers.Embedding(input_dim=n_item,output_dim=dim,
                                             embeddings_regularizer=l2)
    entity_embedding=tf.keras.layers.Embedding(input_dim=n_entity,output_dim=dim,
                                               embeddings_regularizer=l2)
    relation_embedding=tf.keras.layers.Embedding(input_dim=n_relation,output_dim=dim,
                                               embeddings_regularizer=l2)
    u=user_embedding(user_id)
    i=item_embedding(item_id)
    h=entity_embedding(head_id)
    t=entity_embedding(tail_id)
    r=relation_embedding(relation_id)
    for _ in range(L):
        u=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(u)
        i,h=feature_fusion_unit(inputs=(i,h),weight_regularizer=l2)
        t=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(t)

    rs=tf.keras.activations.relu(u*i)
    for _ in range(H-1):
        rs=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(rs)
    rs=tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid,kernel_regularizer=l2)(rs)

    kg = tf.concat([h, r], axis=1)
    for _ in range(H - 1):
        kg = tf.keras.layers.Dense(dim * 2, activation=tf.keras.activations.relu, kernel_regularizer=l2)(kg)
    kg = tf.keras.layers.Dense(dim, activation=tf.keras.activations.sigmoid, kernel_regularizer=l2)(kg)
    kg=-tf.keras.activations.sigmoid(tf.reduce_sum(kg*t,axis=1))
    return tf.keras.Model(inputs=[user_id,item_id,head_id],outputs=rs),tf.keras.Model(inputs=[item_id,head_id,relation_id,tail_id],outputs=kg)

## 训练函数

In [ ]:
def preprocess_rs_data(data):
    u = tf.constant(data[:, 0], dtype=tf.int32)
    i = tf.constant(data[:, 1], dtype=tf.int32)
    h = tf.constant(data[:, 1], dtype=tf.int32)
    r = tf.constant(data[:, 2], dtype=tf.float32)
    return {'user_id': u, 'item_id': i, 'head_id': h}, r


def preprocess_kg_data(data):
    i = tf.constant(data[:, 0], dtype=tf.int32)
    h = tf.constant(data[:, 0], dtype=tf.int32)
    r = tf.constant(data[:, 1], dtype=tf.int32)
    t = tf.constant(data[:, 2], dtype=tf.int32)
    l = tf.zeros(len(data), dtype=tf.float32)
    return {'item_id': i, 'head_id': h, 'relation_id': r, 'tail_id': t}, l


def train(DATASET, DEFAULT):
    n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
    rs_model, kg_model = MFFRM_model(n_user, n_item, n_entity, n_relation, dim=DEFAULT[DATASET]['dim'], H=DEFAULT[DATASET]['H'],L=DEFAULT[DATASET]['L'])
    epochs = DEFAULT[DATASET]['epochs']
    kge_interval = DEFAULT[DATASET]['kge_interval']
    batch = DEFAULT[DATASET]['batch_size']
    rs_train_ds = tf.data.Dataset.from_tensor_slices(preprocess_rs_data(train_data))
    rs_train_ds = rs_train_ds.shuffle(len(train_data)).batch(batch)
    rs_eval_ds = tf.data.Dataset.from_tensor_slices(preprocess_rs_data(eval_data))
    rs_eval_ds = rs_eval_ds.batch(batch)
    kg_train_ds = tf.data.Dataset.from_tensor_slices(preprocess_kg_data(kg))
    kg_train_ds = kg_train_ds.shuffle(len(kg)).batch(batch)
    rs_model.compile(optimizer=tf.optimizers.Adam( DEFAULT[DATASET]['lr_rs']), loss=tf.keras.losses.binary_crossentropy,
                     metrics=['AUC', tf.keras.metrics.binary_accuracy])
    kg_model.compile(optimizer=tf.optimizers.Adam(DEFAULT[DATASET]['lr_kg']), loss=lambda y_true, y_pre: y_pre)
    for epoch in range(epochs):
        history=rs_model.fit(rs_train_ds, epochs=epoch + 1, validation_data=rs_eval_ds, verbose=1, initial_epoch=epoch)
        if epoch % kge_interval == 0:
            kg_model.fit(kg_train_ds, epochs=epoch + 1, initial_epoch=epoch, verbose=1)
        rs_model.save('./log/MFFRM/' + DATASET + '/model_' + str(epoch) + '.h5')
    return history

## 在movielen数据集训练

In [ ]:
DEFAULT={
        'movie': {'epochs': 60, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 4096, 'lr_rs': 0.02, 'lr_kg': 0.01,
                  'kge_interval': 3},
        'book': {'epochs': 10, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 32, 'lr_rs': 2e-4, 'lr_kg': 2e-5,
                 'kge_interval': 2},
        'music': {'epochs': 10, 'dim': 4, 'L': 2, 'H': 1, 'batch_size': 256, 'lr_rs': 1e-3, 'lr_kg': 2e-4,
                  'kge_interval': 2}
    }
DATASET='movie'
tf.random.set_seed(555)
history=train(DATASET,DEFAULT)

reading rating file ....
splitting dataset ...
reading KG file
data loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


5/5 [==============================] - 0s 12ms/step - loss: -0.5267
Epoch 2/2
111/111 [==============================] - 2s 21ms/step - loss: 0.4190 - auc: 0.8893 - binary_accuracy: 0.8061 - val_loss: 0.4256 - val_auc: 0.8871 - val_binary_accuracy: 0.8043
Epoch 3/3
111/111 [==============================] - 2s 21ms/step - loss: 0.4133 - auc: 0.8928 - binary_accuracy: 0.8091 - val_loss: 0.4241 - val_auc: 0.8887 - val_binary_accuracy: 0.8068
Epoch 4/4
111/111 [==============================] - 2s 21ms/step - loss: 0.4057 - auc: 0.8975 - binary_accuracy: 0.8151 - val_loss: 0.4170 - val_auc: 0.8934 - val_binary_accuracy: 0.8137
Epoch 4/4
5/5 [==============================] - 0s 14ms/step - loss: -0.5897
Epoch 5/5
111/111 [==============================] - 2s 20ms/step - loss: 0.3940 - auc: 0.9043 - binary_accuracy: 0.8228 - val_loss: 0.4084 - val_auc: 0.8993 - val_binary_accuracy: 0.8218
Epoch 6/6
111/111 [==============================] - 2s 20ms/step - loss: 0.3810 - auc: 0.9117 - binar

## 在movielen上进行测试

In [ ]:
#测试ctr
n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
network = tf.keras.models.load_model('./log/MFFRM/movie/model_59.h5',custom_objects={'CrossLayer':CrossLayer,'CompressLayer':CompressLayer})
result=network.evaluate([test_data[:,0],test_data[:,1],test_data[:,1]],test_data[:,2])

reading rating file ....
splitting dataset ...
reading KG file
data loaded
4712/4712 [==============================] - 16s 3ms/step - loss: 0.3253 - auc: 0.9428 - binary_accuracy: 0.8733


In [ ]:
print("MFFRM在测试集上auc："+str(result[1]))
print("MFFRM在测试集上acc："+str(result[2]))

MFFRM在测试集上auc：0.9428346157073975
MFFRM在测试集上acc：0.8732902407646179


## 在book-crossing数据集训练

In [ ]:
DATASET='book'
tf.random.set_seed(555)
history=train(DATASET,DEFAULT)

reading rating file ....
splitting dataset ...
reading KG file
data loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


619/619 [==============================] - 3s 5ms/step - loss: -0.5066
Epoch 2/2
2621/2621 [==============================] - 21s 8ms/step - loss: 0.5430 - auc: 0.7945 - binary_accuracy: 0.7360 - val_loss: 0.5776 - val_auc: 0.7428 - val_binary_accuracy: 0.7031
Epoch 3/3
2621/2621 [==============================] - 21s 8ms/step - loss: 0.4990 - auc: 0.8359 - binary_accuracy: 0.7552 - val_loss: 0.6060 - val_auc: 0.7416 - val_binary_accuracy: 0.6986
Epoch 3/3
619/619 [==============================] - 3s 5ms/step - loss: -0.5200
Epoch 4/4
2621/2621 [==============================] - 21s 8ms/step - loss: 0.4744 - auc: 0.8522 - binary_accuracy: 0.7620 - val_loss: 0.6388 - val_auc: 0.7380 - val_binary_accuracy: 0.6973
Epoch 5/5
2621/2621 [==============================] - 21s 8ms/step - loss: 0.4497 - auc: 0.8666 - binary_accuracy: 0.7653 - val_loss: 0.6869 - val_auc: 0.7304 - val_binary_accuracy: 0.6914
Epoch 5/5
619/619 [==============================] - 3s 5ms/step - loss: -0.5347
Epoch 6

## 在book-crossing上测试

In [ ]:
n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
network = tf.keras.models.load_model('./log/MFFRM/book/model_2.h5',custom_objects={'CrossLayer':CrossLayer,'CompressLayer':CompressLayer})
result=network.evaluate([test_data[:,0],test_data[:,1],test_data[:,1]],test_data[:,2])
print("MFFRM在测试集上auc："+str(result[1]))
print("MFFRM在测试集上acc："+str(result[2]))

reading rating file ....
splitting dataset ...
reading KG file
data loaded
874/874 [==============================] - 3s 4ms/step - loss: 0.5149 - auc_3: 0.8216 - binary_accuracy: 0.7468
MFFRM在测试集上auc：0.8215720653533936
MFFRM在测试集上acc：0.7467887997627258


## 在last.FM数据集上训练

In [ ]:
DEFAULT={
        'movie': {'epochs': 60, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 4096, 'lr_rs': 0.02, 'lr_kg': 0.01,
                  'kge_interval': 3},
        'book': {'epochs': 10, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 32, 'lr_rs': 2e-4, 'lr_kg': 2e-5,
                 'kge_interval': 2},
        'music': {'epochs': 20, 'dim': 4, 'L': 2, 'H': 1, 'batch_size': 128, 'lr_rs': 1e-4, 'lr_kg': 2e-4,
                  'kge_interval': 2}
    }
DATASET='music'
tf.random.set_seed(555)
history=train(DATASET,DEFAULT)

reading rating file ....
splitting dataset ...
reading KG file
data loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


122/122 [==============================] - 1s 5ms/step - loss: -0.5104
Epoch 2/2
199/199 [==============================] - 2s 8ms/step - loss: 0.6932 - auc: 0.5003 - binary_accuracy: 0.5026 - val_loss: 0.6931 - val_auc: 0.5027 - val_binary_accuracy: 0.4965
Epoch 3/3
199/199 [==============================] - 2s 8ms/step - loss: 0.6925 - auc: 0.6063 - binary_accuracy: 0.5134 - val_loss: 0.6913 - val_auc: 0.6733 - val_binary_accuracy: 0.5898
Epoch 3/3
122/122 [==============================] - 1s 5ms/step - loss: -0.5379
Epoch 4/4
199/199 [==============================] - 2s 8ms/step - loss: 0.6878 - auc: 0.7242 - binary_accuracy: 0.5100 - val_loss: 0.6825 - val_auc: 0.7446 - val_binary_accuracy: 0.5251
Epoch 5/5
199/199 [==============================] - 2s 8ms/step - loss: 0.6592 - auc: 0.8050 - binary_accuracy: 0.6751 - val_loss: 0.6379 - val_auc: 0.7721 - val_binary_accuracy: 0.6887
Epoch 5/5
122/122 [==============================] - 1s 5ms/step - loss: -0.5829
Epoch 6/6
199/199 [

## 在last.FM上测试

In [ ]:
n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
network = tf.keras.models.load_model('./log/MFFRM/music/model_18.h5',custom_objects={'CrossLayer':CrossLayer,'CompressLayer':CompressLayer})
result=network.evaluate([test_data[:,0],test_data[:,1],test_data[:,1]],test_data[:,2])
print("MFFRM在测试集上auc："+str(result[1]))
print("MFFRM在测试集上acc："+str(result[2]))

reading rating file ....
splitting dataset ...
reading KG file
data loaded
265/265 [==============================] - 1s 4ms/step - loss: 0.4933 - auc_7: 0.8466 - binary_accuracy: 0.7667
MFFRM在测试集上auc：0.8465502262115479
MFFRM在测试集上acc：0.7666784524917603
